In [1]:
import keras,os
from keras.models import Sequential #so that all layers are arranged in sequence
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [2]:
train_dir = "/kaggle/input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_images"
datagen = ImageDataGenerator(validation_split=0.2)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')

val_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')


# val_generator = datagen.flow_from_directory(
#     train_dir,
#     target_size=(224, 224),
#     batch_size=32,
#     class_mode='categorical',
#     subset='validation',
#     shuffle=False)

# # Obtain the validation features and labels
# val_features, val_labels = val_generator.next()


Found 37297 images belonging to 3116 classes.
Found 7405 images belonging to 3116 classes.


#CNN model

In [3]:
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
#Add fully connected layers
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=3116, activation="softmax"))
#Show the structure of the CNN
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                    

# Optimization to reach global minima during training

In [4]:
from keras.optimizers import Adam
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

#Monitoring validation accuracy
The model will only be saved to disk if the validation accuracy of the model in current epoch is greater than what it was in the last epoch.

In [5]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq=1)
#early = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit(train_generator, epochs=150, steps_per_epoch=100, validation_data=val_generator, validation_steps=10)


# hist = model.fit(train_generator, epochs=1, steps_per_epoch=2, validation_data=val_generator, validation_steps=10, callbacks=[checkpoint, early])
# for key in hist.history:
#   print(key)
#hist = model.fit(train_generator, epochs=100, validation_data=(val_features, val_labels), callbacks=[checkpoint, early])

#hist = model.fit(train_generator, epochs=100, steps_per_epoch=100, validation_data=(val_features, val_labels), callbacks=[checkpoint, early])

#hist = model.fit(train_generator, epochs=10, steps_per_epoch=8, validation_data=(val_features, val_labels), callbacks=[checkpoint, early])

# hist = model.fit(train_generator, epochs=10, steps_per_epoch=100, validation_data=val_generator, validation_steps=10, callbacks=[checkpoint, early])
# hist = model.fit_generator(steps_per_epoch=8,generator=train_generator, validation_data= val_generator, validation_steps=10,epochs=10,callbacks=[checkpoint,early])

Epoch 1/150
100/100 [==============================] - 253s 2s/step - loss: 40.7565 - accuracy: 0.0156 - val_loss: 7.6686 - val_accuracy: 0.0437
Epoch 2/150
100/100 [==============================] - 241s 2s/step - loss: 7.7080 - accuracy: 0.0253 - val_loss: 7.5397 - val_accuracy: 0.0312
Epoch 3/150
100/100 [==============================] - 234s 2s/step - loss: 7.6999 - accuracy: 0.0305 - val_loss: 7.3252 - val_accuracy: 0.0312
Epoch 4/150
100/100 [==============================] - 231s 2s/step - loss: 7.6379 - accuracy: 0.0231 - val_loss: 7.4023 - val_accuracy: 0.0500
Epoch 5/150
100/100 [==============================] - 236s 2s/step - loss: 7.5596 - accuracy: 0.0366 - val_loss: 7.2221 - val_accuracy: 0.0375
Epoch 6/150
100/100 [==============================] - 227s 2s/step - loss: 7.6026 - accuracy: 0.0284 - val_loss: 7.3906 - val_accuracy: 0.0312
Epoch 7/150
100/100 [==============================] - 224s 2s/step - loss: 7.5961 - accuracy: 0.0267 - val_loss: 7.2681 - val_accuracy

**Save the model**

In [6]:
model.save("try_model.h5")


**Saving as json file**

In [7]:
from keras.models import model_from_json
# serialize model to json
json_model = model.to_json()
#save the model architecture to JSON file
with open('fashionmnist_model.json', 'w') as json_file:
    json_file.write(json_model)
#saving the weights of the model
model.save_weights('FashionMNIST_weights.h5')
#Model loss and accuracy
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)

NameError: name 'test_images' is not defined

**Creating zip folder**

In [ ]:
!zip -r file.zip "/kaggle/working/try_model.h5"

**Load the model**

In [ ]:
from tensorflow.keras.models import load_model
loaded_model = load_model("try_model.h5")
loaded_model.summary()

**Removing unwanted files**

In [ ]:
#os.remove("/kaggle/working/file.zip")

#Visualise training/validation accuracy and loss

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history["accuracy"])
plt.plot(hist.history['val_accuracy'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()